In [1]:
import pandas as pd
import numpy as np
import json
import warnings
import spacy
warnings.filterwarnings("ignore")
import random

import nltk
from nltk.stem import WordNetLemmatizer

import re

from spacy.tokens import DocBin

from tqdm import tqdm

In [2]:
train_data = pd.read_csv(r"C:\Users\sai\Desktop\PRAXIS\Term - 3\CAPP\Datasets\train.csv")
features = pd.read_csv(r"C:\Users\sai\Desktop\PRAXIS\Term - 3\CAPP\Datasets\features.csv")
patient_notes = pd.read_csv(r"C:\Users\sai\Desktop\PRAXIS\Term - 3\CAPP\Datasets\patient_notes.csv")

In [3]:
new_train = train_data[(train_data.feature_num == 0)][['pn_num','annotation',"location","feature_num"]]
new_train

,pn_num,annotation,location,feature_num
0,16,['dad with recent heart attcak'],['696 724'],0
13,41,[],[],0
26,46,['father: heart attack'],['824 844'],0
39,82,['Father MI'],['622 631'],0
52,100,['Dad-MI'],['735 741'],0
...,...,...,...,...
1235,2315,['father MI'],['750 756;761 763'],0
1248,2425,[],[],0
1261,2428,['father heart attack'],['589 595;606 618'],0
1274,2430,['MI father'],['845 847;855 861'],0


In [4]:
new_train = new_train[new_train.annotation != '[]']
new_train

,pn_num,annotation,location,feature_num
0,16,['dad with recent heart attcak'],['696 724'],0
26,46,['father: heart attack'],['824 844'],0
39,82,['Father MI'],['622 631'],0
52,100,['Dad-MI'],['735 741'],0
65,161,['father had acute MI'],['601 620'],0
...,...,...,...,...
1222,2308,['Dad had MI'],['909 919'],0
1235,2315,['father MI'],['750 756;761 763'],0
1261,2428,['father heart attack'],['589 595;606 618'],0
1274,2430,['MI father'],['845 847;855 861'],0


In [5]:
new_train["annotation"] = new_train.annotation.str.replace('[','')
new_train["annotation"] = new_train.annotation.str.replace(']','')
new_train["location"] = new_train.location.str.replace('[','')
new_train["location"] = new_train.location.str.replace(']','')

new_train

,pn_num,annotation,location,feature_num
0,16,'dad with recent heart attcak','696 724',0
26,46,'father: heart attack','824 844',0
39,82,'Father MI','622 631',0
52,100,'Dad-MI','735 741',0
65,161,'father had acute MI','601 620',0
...,...,...,...,...
1222,2308,'Dad had MI','909 919',0
1235,2315,'father MI','750 756;761 763',0
1261,2428,'father heart attack','589 595;606 618',0
1274,2430,'MI father','845 847;855 861',0


In [6]:
new_train.loc[new_train["annotation"] == "'dad, heart attack'","annotation"] = "'dad  heart attack'"
new_train

,pn_num,annotation,location,feature_num
0,16,'dad with recent heart attcak','696 724',0
26,46,'father: heart attack','824 844',0
39,82,'Father MI','622 631',0
52,100,'Dad-MI','735 741',0
65,161,'father had acute MI','601 620',0
...,...,...,...,...
1222,2308,'Dad had MI','909 919',0
1235,2315,'father MI','750 756;761 763',0
1261,2428,'father heart attack','589 595;606 618',0
1274,2430,'MI father','845 847;855 861',0


In [7]:
new_train = (new_train.set_index(['pn_num',"feature_num"]) 
   .apply(lambda col: col.str.split(',').explode())
   .reset_index()
   .reindex(new_train.columns, axis=1))
new_train

,pn_num,annotation,location,feature_num
0,16,'dad with recent heart attcak','696 724',0
1,46,'father: heart attack','824 844',0
2,82,'Father MI','622 631',0
3,100,'Dad-MI','735 741',0
4,161,'father had acute MI','601 620',0
...,...,...,...,...
87,2315,'father MI','750 756;761 763',0
88,2428,'father heart attack','589 595;606 618',0
89,2430,'MI father','845 847;855 861',0
90,2436,'Father MI','915 924',0


In [8]:
new_train["start_location"] = new_train["location"].apply(lambda x: x.split()[0][1:])
new_train["end_location"] = new_train["location"].apply(lambda x: x.split()[-1][:-1])
new_train

,pn_num,annotation,location,feature_num,start_location,end_location
0,16,'dad with recent heart attcak','696 724',0,696,724
1,46,'father: heart attack','824 844',0,824,844
2,82,'Father MI','622 631',0,622,631
3,100,'Dad-MI','735 741',0,735,741
4,161,'father had acute MI','601 620',0,601,620
...,...,...,...,...,...,...
87,2315,'father MI','750 756;761 763',0,750,763
88,2428,'father heart attack','589 595;606 618',0,589,618
89,2430,'MI father','845 847;855 861',0,845,861
90,2436,'Father MI','915 924',0,915,924


In [9]:
new_train = new_train.merge(patient_notes, on='pn_num', how='left')
new_train

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history
0,16,'dad with recent heart attcak','696 724',0,696,724,0,HPI: 17yo M presents with palpitations. Patien...
1,46,'father: heart attack','824 844',0,824,844,0,Mr. Cleveland is a 17yo M who was consented by...
2,82,'Father MI','622 631',0,622,631,0,17 yo M w/ no cardiac or arrhythmia PMH presen...
3,100,'Dad-MI','735 741',0,735,741,0,HPI: Dillon Cleveland is an otherwise healthy ...
4,161,'father had acute MI','601 620',0,601,620,0,"17 y/o M , Dillon Cleveland comes with c/o of ..."
...,...,...,...,...,...,...,...,...
87,2315,'father MI','750 756;761 763',0,750,763,0,17 yo M who presents with episodes of heart pa...
88,2428,'father heart attack','589 595;606 618',0,589,618,0,dillon cleveland is a 17yo male with no PMH th...
89,2430,'MI father','845 847;855 861',0,845,861,0,Mr. Cleveland is a 17 yo male who presents wit...
90,2436,'Father MI','915 924',0,915,924,0,CC: Dillon Cleveland is a 17 year old male wit...


In [10]:
new_train["New_annotation"] = new_train.apply(lambda x: x.pn_history[int(x.start_location):int(x.end_location)],axis=1)

In [11]:
new_train["New_annotation"] = new_train["New_annotation"].apply(lambda x: re.sub(r'''[/",()\r\n]''',' ',x))
new_train["New_annotation"] = new_train["New_annotation"].apply(lambda x: re.sub(r'''[']''','',x))
new_train["New_annotation"] = new_train["New_annotation"].apply(lambda x: x.strip())
new_train

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history,New_annotation
0,16,'dad with recent heart attcak','696 724',0,696,724,0,HPI: 17yo M presents with palpitations. Patien...,dad with recent heart attcak
1,46,'father: heart attack','824 844',0,824,844,0,Mr. Cleveland is a 17yo M who was consented by...,father: heart attack
2,82,'Father MI','622 631',0,622,631,0,17 yo M w/ no cardiac or arrhythmia PMH presen...,Father MI
3,100,'Dad-MI','735 741',0,735,741,0,HPI: Dillon Cleveland is an otherwise healthy ...,Dad-MI
4,161,'father had acute MI','601 620',0,601,620,0,"17 y/o M , Dillon Cleveland comes with c/o of ...",father had acute MI
...,...,...,...,...,...,...,...,...,...
87,2315,'father MI','750 756;761 763',0,750,763,0,17 yo M who presents with episodes of heart pa...,father had MI
88,2428,'father heart attack','589 595;606 618',0,589,618,0,dillon cleveland is a 17yo male with no PMH th...,father who had a heart attack
89,2430,'MI father','845 847;855 861',0,845,861,0,Mr. Cleveland is a 17 yo male who presents wit...,MI in his father
90,2436,'Father MI','915 924',0,915,924,0,CC: Dillon Cleveland is a 17 year old male wit...,Father MI


In [12]:
new_train["New_pn_history"] = new_train["pn_history"].apply(lambda x: re.sub(r'''[/",()\r\n]''',' ',x))
new_train["New_pn_history"] = new_train["New_pn_history"].apply(lambda x: re.sub(r'''[']''','',x))
new_train

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history,New_annotation,New_pn_history
0,16,'dad with recent heart attcak','696 724',0,696,724,0,HPI: 17yo M presents with palpitations. Patien...,dad with recent heart attcak,HPI: 17yo M presents with palpitations. Patien...
1,46,'father: heart attack','824 844',0,824,844,0,Mr. Cleveland is a 17yo M who was consented by...,father: heart attack,Mr. Cleveland is a 17yo M who was consented by...
2,82,'Father MI','622 631',0,622,631,0,17 yo M w/ no cardiac or arrhythmia PMH presen...,Father MI,17 yo M w no cardiac or arrhythmia PMH presen...
3,100,'Dad-MI','735 741',0,735,741,0,HPI: Dillon Cleveland is an otherwise healthy ...,Dad-MI,HPI: Dillon Cleveland is an otherwise healthy ...
4,161,'father had acute MI','601 620',0,601,620,0,"17 y/o M , Dillon Cleveland comes with c/o of ...",father had acute MI,17 y o M Dillon Cleveland comes with c o of ...
...,...,...,...,...,...,...,...,...,...,...
87,2315,'father MI','750 756;761 763',0,750,763,0,17 yo M who presents with episodes of heart pa...,father had MI,17 yo M who presents with episodes of heart pa...
88,2428,'father heart attack','589 595;606 618',0,589,618,0,dillon cleveland is a 17yo male with no PMH th...,father who had a heart attack,dillon cleveland is a 17yo male with no PMH th...
89,2430,'MI father','845 847;855 861',0,845,861,0,Mr. Cleveland is a 17 yo male who presents wit...,MI in his father,Mr. Cleveland is a 17 yo male who presents wit...
90,2436,'Father MI','915 924',0,915,924,0,CC: Dillon Cleveland is a 17 year old male wit...,Father MI,CC: Dillon Cleveland is a 17 year old male wit...


In [13]:
def word_lemm(sentence):
    word_list = nltk.word_tokenize(sentence)
    
    lemmaztier = WordNetLemmatizer()
    lemmatized_output = ' '.join([lemmaztier.lemmatize(w) for w in word_list])
    return(lemmatized_output)

In [14]:
new_train["New_annotation"] = new_train["New_annotation"].apply(word_lemm)
new_train["New_pn_history"] = new_train["New_pn_history"].apply(word_lemm)
new_train.head()

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history,New_annotation,New_pn_history
0,16,'dad with recent heart attcak','696 724',0,696,724,0,HPI: 17yo M presents with palpitations. Patien...,dad with recent heart attcak,HPI : 17yo M present with palpitation . Patien...
1,46,'father: heart attack','824 844',0,824,844,0,Mr. Cleveland is a 17yo M who was consented by...,father : heart attack,Mr. Cleveland is a 17yo M who wa consented by ...
2,82,'Father MI','622 631',0,622,631,0,17 yo M w/ no cardiac or arrhythmia PMH presen...,Father MI,17 yo M w no cardiac or arrhythmia PMH present...
3,100,'Dad-MI','735 741',0,735,741,0,HPI: Dillon Cleveland is an otherwise healthy ...,Dad-MI,HPI : Dillon Cleveland is an otherwise healthy...
4,161,'father had acute MI','601 620',0,601,620,0,"17 y/o M , Dillon Cleveland comes with c/o of ...",father had acute MI,17 y o M Dillon Cleveland come with c o of pal...


In [15]:
new_train["new_location"] = new_train.apply(lambda x :re.search(r'\b' + x.New_annotation + r'\b', x.New_pn_history),axis =1)
new_train

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history,New_annotation,New_pn_history,new_location
0,16,'dad with recent heart attcak','696 724',0,696,724,0,HPI: 17yo M presents with palpitations. Patien...,dad with recent heart attcak,HPI : 17yo M present with palpitation . Patien...,"<re.Match object; span=(663, 691), match='dad ..."
1,46,'father: heart attack','824 844',0,824,844,0,Mr. Cleveland is a 17yo M who was consented by...,father : heart attack,Mr. Cleveland is a 17yo M who wa consented by ...,"<re.Match object; span=(797, 818), match='fath..."
2,82,'Father MI','622 631',0,622,631,0,17 yo M w/ no cardiac or arrhythmia PMH presen...,Father MI,17 yo M w no cardiac or arrhythmia PMH present...,"<re.Match object; span=(598, 607), match='Fath..."
3,100,'Dad-MI','735 741',0,735,741,0,HPI: Dillon Cleveland is an otherwise healthy ...,Dad-MI,HPI : Dillon Cleveland is an otherwise healthy...,"<re.Match object; span=(701, 707), match='Dad-..."
4,161,'father had acute MI','601 620',0,601,620,0,"17 y/o M , Dillon Cleveland comes with c/o of ...",father had acute MI,17 y o M Dillon Cleveland come with c o of pal...,"<re.Match object; span=(589, 608), match='fath..."
...,...,...,...,...,...,...,...,...,...,...,...
87,2315,'father MI','750 756;761 763',0,750,763,0,17 yo M who presents with episodes of heart pa...,father had MI,17 yo M who present with episode of heart palp...,"<re.Match object; span=(726, 739), match='fath..."
88,2428,'father heart attack','589 595;606 618',0,589,618,0,dillon cleveland is a 17yo male with no PMH th...,father who had a heart attack,dillon cleveland is a 17yo male with no PMH th...,"<re.Match object; span=(573, 602), match='fath..."
89,2430,'MI father','845 847;855 861',0,845,861,0,Mr. Cleveland is a 17 yo male who presents wit...,MI in his father,Mr. Cleveland is a 17 yo male who present with...,"<re.Match object; span=(823, 839), match='MI i..."
90,2436,'Father MI','915 924',0,915,924,0,CC: Dillon Cleveland is a 17 year old male wit...,Father MI,CC : Dillon Cleveland is a 17 year old male wi...,"<re.Match object; span=(898, 907), match='Fath..."


In [16]:
drop_index = []
for  index,row in new_train.iterrows():
    if row["new_location"] == None:
        drop_index.append(index)
print(drop_index)

[26]


In [17]:
new_train = new_train.drop(drop_index,axis = 0)
new_train

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history,New_annotation,New_pn_history,new_location
0,16,'dad with recent heart attcak','696 724',0,696,724,0,HPI: 17yo M presents with palpitations. Patien...,dad with recent heart attcak,HPI : 17yo M present with palpitation . Patien...,"<re.Match object; span=(663, 691), match='dad ..."
1,46,'father: heart attack','824 844',0,824,844,0,Mr. Cleveland is a 17yo M who was consented by...,father : heart attack,Mr. Cleveland is a 17yo M who wa consented by ...,"<re.Match object; span=(797, 818), match='fath..."
2,82,'Father MI','622 631',0,622,631,0,17 yo M w/ no cardiac or arrhythmia PMH presen...,Father MI,17 yo M w no cardiac or arrhythmia PMH present...,"<re.Match object; span=(598, 607), match='Fath..."
3,100,'Dad-MI','735 741',0,735,741,0,HPI: Dillon Cleveland is an otherwise healthy ...,Dad-MI,HPI : Dillon Cleveland is an otherwise healthy...,"<re.Match object; span=(701, 707), match='Dad-..."
4,161,'father had acute MI','601 620',0,601,620,0,"17 y/o M , Dillon Cleveland comes with c/o of ...",father had acute MI,17 y o M Dillon Cleveland come with c o of pal...,"<re.Match object; span=(589, 608), match='fath..."
...,...,...,...,...,...,...,...,...,...,...,...
87,2315,'father MI','750 756;761 763',0,750,763,0,17 yo M who presents with episodes of heart pa...,father had MI,17 yo M who present with episode of heart palp...,"<re.Match object; span=(726, 739), match='fath..."
88,2428,'father heart attack','589 595;606 618',0,589,618,0,dillon cleveland is a 17yo male with no PMH th...,father who had a heart attack,dillon cleveland is a 17yo male with no PMH th...,"<re.Match object; span=(573, 602), match='fath..."
89,2430,'MI father','845 847;855 861',0,845,861,0,Mr. Cleveland is a 17 yo male who presents wit...,MI in his father,Mr. Cleveland is a 17 yo male who present with...,"<re.Match object; span=(823, 839), match='MI i..."
90,2436,'Father MI','915 924',0,915,924,0,CC: Dillon Cleveland is a 17 year old male wit...,Father MI,CC : Dillon Cleveland is a 17 year old male wi...,"<re.Match object; span=(898, 907), match='Fath..."


In [18]:
new_train["start_location"] = new_train.apply(lambda x :re.search(r'\b' + x.New_annotation + r'\b', x.New_pn_history).start(),axis =1)
new_train["end_location"] = new_train.apply(lambda x :re.search(r'\b' + x.New_annotation + r'\b', x.New_pn_history).end(),axis =1)
new_train

,pn_num,annotation,location,feature_num,start_location,end_location,case_num,pn_history,New_annotation,New_pn_history,new_location
0,16,'dad with recent heart attcak','696 724',0,663,691,0,HPI: 17yo M presents with palpitations. Patien...,dad with recent heart attcak,HPI : 17yo M present with palpitation . Patien...,"<re.Match object; span=(663, 691), match='dad ..."
1,46,'father: heart attack','824 844',0,797,818,0,Mr. Cleveland is a 17yo M who was consented by...,father : heart attack,Mr. Cleveland is a 17yo M who wa consented by ...,"<re.Match object; span=(797, 818), match='fath..."
2,82,'Father MI','622 631',0,598,607,0,17 yo M w/ no cardiac or arrhythmia PMH presen...,Father MI,17 yo M w no cardiac or arrhythmia PMH present...,"<re.Match object; span=(598, 607), match='Fath..."
3,100,'Dad-MI','735 741',0,701,707,0,HPI: Dillon Cleveland is an otherwise healthy ...,Dad-MI,HPI : Dillon Cleveland is an otherwise healthy...,"<re.Match object; span=(701, 707), match='Dad-..."
4,161,'father had acute MI','601 620',0,589,608,0,"17 y/o M , Dillon Cleveland comes with c/o of ...",father had acute MI,17 y o M Dillon Cleveland come with c o of pal...,"<re.Match object; span=(589, 608), match='fath..."
...,...,...,...,...,...,...,...,...,...,...,...
87,2315,'father MI','750 756;761 763',0,726,739,0,17 yo M who presents with episodes of heart pa...,father had MI,17 yo M who present with episode of heart palp...,"<re.Match object; span=(726, 739), match='fath..."
88,2428,'father heart attack','589 595;606 618',0,573,602,0,dillon cleveland is a 17yo male with no PMH th...,father who had a heart attack,dillon cleveland is a 17yo male with no PMH th...,"<re.Match object; span=(573, 602), match='fath..."
89,2430,'MI father','845 847;855 861',0,823,839,0,Mr. Cleveland is a 17 yo male who presents wit...,MI in his father,Mr. Cleveland is a 17 yo male who present with...,"<re.Match object; span=(823, 839), match='MI i..."
90,2436,'Father MI','915 924',0,898,907,0,CC: Dillon Cleveland is a 17 year old male wit...,Father MI,CC : Dillon Cleveland is a 17 year old male wi...,"<re.Match object; span=(898, 907), match='Fath..."


In [19]:
sep_combined_case0  = new_train.copy()

In [20]:
row_index = []
start_location = []
end_location = []
pn_notes = set()
drop_index = []
for index,row in sep_combined_case0.iterrows():   # Iterating every row in trainn data
    if row["pn_num"] not in pn_notes:    # Verifying whether there is a change in pn_num to reinitialize start and end location lists
        start_location = []
        end_location = []
        pn_notes.add(row["pn_num"])      # Appending pn_num
        if (row["start_location"] in start_location) | (row["end_location"] in end_location):
            row_index.append(index)
        start_location.append(row["start_location"])
        end_location.append(row["end_location"])
    else: 
        if (row["start_location"] in start_location) | (row["end_location"] in end_location):
            row_index.append(index)
            
            if row["start_location"] in start_location:
                if row["end_location"] >= sep_combined_case0.iloc[index -1 ,5]:
                    drop_index.append(index -1)
                else:
                    drop_index.append(index)
            elif row["end_location"] in end_location:
                if row["start_location"] <= sep_combined_case0.iloc[index -1,4]:
                    drop_index.append(index - 1)
                else:
                    drop_index.append(index)
        start_location.append(row["start_location"])
        end_location.append(row["end_location"])            

In [21]:
print(drop_index)

[14]


In [22]:
sep_combined_case0 = sep_combined_case0.drop(index=drop_index)

In [23]:
pn_Num_list = list(sep_combined_case0.pn_num.unique())

In [24]:
final_train = []   # appending all the individual rows
for patient_number in pn_Num_list:  # for each unique patient number
    entities = []  # saving individaual entities locations
    for row_index,row in sep_combined_case0[sep_combined_case0.pn_num == patient_number].iterrows():  # for each unique patient number 
        
        entities.append((int(row["start_location"]),int(row["end_location"]),"Feature_" + str(row["feature_num"]))) # store the locations in numeric format
        
    text = sep_combined_case0[sep_combined_case0.pn_num == patient_number]["New_pn_history"].values[0] # save the text for the unique patient number
    
    final_train.append((text,{"entities":entities}))  # for each unique pn_number append to final list

In [25]:
final_train[6]

('HPI : Patient is a 17 yo m with a c o of palpitation . Palpitations began a few month ago . States that palpitation are sudden unpredictable and feel like his heart is pounding fast jumping out of his chest . Typically these episode last 3-4 minute and resolve on their own . His most recent episode wa 2 day ago and lasted about 10 minute . During this epsiode he felt lightheaded Short of breath and had chest pressure located in the middle of his chest . Denies any sweating change in hair or bowel movement . ROS : Negative except a stated above PMH : None Meds : Takes his roommate Adderall to help study Allergies : NKDA PSHx : None FH : Mother ha a thyroid problem Father had a MI this past year at age 53 SH : denies to',
 {'entities': [(672, 687, 'Feature_0')]})

In [26]:
# Serializing json 
json_object = json.dumps(final_train, indent = 4)
  
# Writing to sample.json
with open("sample.json", "w") as outfile:
    outfile.write(json_object)

In [27]:
# Opening JSON file
f = open("sample.json")
 
# returns JSON object as
# a dictionary
data = json.load(f)

In [28]:
nlp = spacy.blank("en")
def create_training(TRAIN_DATA):
    db = DocBin()
    for text, annot in tqdm(TRAIN_DATA):
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print ("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return (db)

In [29]:
train = data[:int(len(data)*0.8)]
validation = data[int(len(data)*0.8):]

In [30]:
len(train),len(validation)

(69, 18)

In [31]:
#train data set
camp_train = create_training(train)
camp_train.to_disk("nbme_train.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 69/69 [00:00<00:00, 907.08it/s]


In [32]:
# Validation data set
camp_train = create_training(validation)
camp_train.to_disk("nbme_validation.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 947.15it/s]


In [33]:
!python -m spacy init fill-config ./base_config.cfg ./config.cfg

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-04-05 23:46:07.854900: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-04-05 23:46:07.854939: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [34]:
!python -m spacy train config.cfg --output ./output

[+] Created output directory: output
[i] Saving to output directory: output
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     55.50    0.00    0.00    0.00    0.00
  2     200        121.68   1299.83   78.95   78.95   78.95    0.79
  5     400         18.45     49.23   84.21   84.21   84.21    0.84
  8     600         17.05     25.09   84.21   84.21   84.21    0.84
 11     800          6.28      5.56   91.89   94.44   89.47    0.92
 14    1000          6.46      5.90   84.21   84.21   84.21    0.84
 17    1200      15849.11    106.58   75.68   77.78   73.68    0.76
 20    1400          6.92      5.55   71.79   70.00 

2022-04-05 23:46:11.351451: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-04-05 23:46:11.351490: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-04-05 23:46:13,587] [INFO] Set up nlp object from config
[2022-04-05 23:46:13,593] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-04-05 23:46:13,595] [INFO] Created vocabulary
[2022-04-05 23:46:13,596] [INFO] Finished initializing nlp object
[2022-04-05 23:46:13,931] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
